## Relax 
##### Reference: GitHub: olsenben/relax-data-science-challenge

In [2]:
import pandas as pd
users = pd.read_csv('takehome_users.csv', encoding = "ISO-8859-1", parse_dates=True)
user_engage = pd.read_csv('takehome_user_engagement.csv', encoding = "ISO-8859-1", parse_dates=True)

In [3]:
user_engage.head(3)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1


In [4]:
users.head(3)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0


In [5]:
# Look for missing values
missing = pd.concat([users.isnull().sum(), 100 * users.isnull().mean()],axis = 1)
missing.columns=['count', '%']
missing = missing.sort_values(by='count')
missing

,count,%
object_id,0,0.000
creation_time,0,0.000
name,0,0.000
email,0,0.000
creation_source,0,0.000
opted_in_to_mailing_list,0,0.000
enabled_for_marketing_drip,0,0.000
org_id,0,0.000
last_session_creation_time,3177,26.475
invited_by_user_id,5583,46.525


In [6]:
# fix the "last_session_creation_time"
users['last_session_creation_time']= users['last_session_creation_time'].fillna(users['last_session_creation_time'].median())

# Look for missing values in user_engage
missing = pd.concat([user_engage.isnull().sum(), 100 * user_engage.isnull().mean()],axis = 1)
missing.columns=['count', '%']
missing = missing.sort_values(by='count')
missing

,count,%
time_stamp,0,0.0
user_id,0,0.0
visited,0,0.0


In [7]:
users.dtypes

object_id                       int64
creation_time                  object
name                           object
email                          object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
dtype: object

In [8]:
# Look for missing values
missing = pd.concat([users.isnull().sum(), 100 * users.isnull().mean()],axis = 1)
missing.columns=['count', '%']
missing = missing.sort_values(by='count')
missing

,count,%
object_id,0,0.000
creation_time,0,0.000
name,0,0.000
email,0,0.000
creation_source,0,0.000
last_session_creation_time,0,0.000
opted_in_to_mailing_list,0,0.000
enabled_for_marketing_drip,0,0.000
org_id,0,0.000
invited_by_user_id,5583,46.525


In [9]:
user_engage.dtypes

time_stamp    object
user_id        int64
visited        int64
dtype: object

In [10]:
from pandas_profiling import ProfileReport
profile = ProfileReport(users, title='Pandas Profiling Report', explorative=True)
profile.to_widgets()
profile.to_notebook_iframe()

### Label Users as adopted

In [11]:
import datetime

#set the time_stamp to datetime and the set it as the index
user_engage.time_stamp = pd.to_datetime(user_engage.time_stamp)
user_engage = user_engage.set_index('time_stamp', drop= True)

In [12]:
from datetime import timedelta

def label_adopted(x):    
    "takes a users input and returns whether or not they have been active within any 7-day period"
    df_temp = user_engage.loc[user_engage['user_id'] == x] #select out rows of this user
    df_temp = df_temp.resample('D').mean().dropna() #resample to show if active in a day. .mean() is just of 1
    adopted = 0
    for i in range(len(df_temp)-2): #loop over active days till the second to last day
        if df_temp.index[i + 2] - df_temp.index[i] <= timedelta(days=7): # difference between every 1st and 3rd day
            adopted = 1
            break
        else:
            adopted = 0
    return adopted

In [13]:
#apply to user df to label users as adopted=true 
users['adopted_user'] = users['object_id'].apply(label_adopted)

In [14]:
print(sum(users['adopted_user']))
print(sum(users.adopted_user)/len(users.adopted_user))

1656
0.138


There are only 1656 active users in the dataframe: it means that the classes are unbalanced. Need to fix.

### Data Wrangling

In [15]:
# Look for missing values
missing = pd.concat([users.isnull().sum(), 100 * users.isnull().mean()],axis = 1)
missing.columns=['count', '%']
missing = missing.sort_values(by='count')
missing

,count,%
object_id,0,0.000
creation_time,0,0.000
name,0,0.000
email,0,0.000
creation_source,0,0.000
last_session_creation_time,0,0.000
opted_in_to_mailing_list,0,0.000
enabled_for_marketing_drip,0,0.000
org_id,0,0.000
adopted_user,0,0.000


#### Convert all time stamps to datetime objects.

In [16]:
users.dtypes
# set creation_time
users.creation_time = pd.to_datetime(users.creation_time)
#last_session_creation_time is a unix time stamp, so use map to convert to datetime
users['last_session_creation_time'] = pd.to_datetime(users.last_session_creation_time)
#users['last_session_creation_time'] = users['last_session_creation_time'].map(lambda data: datetime.datetime.fromtimestamp(int(data)).strftime('%Y-%m-%d %H:%M:%S'), na_action='ignore')

#### Feature usage_length that basically indicates how long a user has been active:

In [17]:
#now set that to datetime
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'])
users['creation_time'] = pd.to_datetime(users['creation_time'])
#subtract to find time active
users['usage_length'] = users['last_session_creation_time'] - users['creation_time']
#lets settle for seconds instead of days to make the time differences more distinct
users['usage_length'] = [x.total_seconds() for x in users['usage_length']]

In [18]:
# Impute missing values: Features look skewed, so we'll impute missing values with median and mean for non-skewed
#users['last_session_creation_time'] = users['last_session_creation_time'].fillna(users['last_session_creation_time'].median())
users['invited_by_user_id'] = users['invited_by_user_id'].fillna(users['invited_by_user_id'].mean())

In [19]:
users['email_provider'] = [x.split('@')[1] for x in users.email]#select out the domain
top_emails = users.email_provider.value_counts().index[:6]
#label anything not in the top 5 as other
users['email_provider'] = [x if x in top_emails else 'other' for x in users.email_provider]

In [20]:
from sklearn.preprocessing import LabelEncoder

gle = LabelEncoder()

creation_labels = gle.fit_transform(users['creation_source'])
users.creation_source = creation_labels

gle2 = LabelEncoder()
email_labels = gle2.fit_transform(users['email_provider'])
users.email_provider = email_labels

# Need to drop 'name', 'creation_time', 'last_session_creation_time', 'invited_by_user_id'
users

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,usage_length,email_provider
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,0,1970-01-01 00:00:01.398138810,1,0,11,10803.000000,0,-1.398139e+09,6
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,1,1970-01-01 00:00:01.396237504,0,0,1,316.000000,1,-1.384487e+09,2
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,1,1970-01-01 00:00:01.363734892,0,0,94,1525.000000,0,-1.363735e+09,2
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,0,1970-01-01 00:00:01.369210168,0,0,1,5151.000000,0,-1.369124e+09,6
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,0,1970-01-01 00:00:01.358849660,0,0,193,5240.000000,0,-1.358418e+09,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,1,1970-01-01 00:00:01.378448055,0,0,89,8263.000000,0,-1.378448e+09,2
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,4,1970-01-01 00:00:01.358274517,0,0,200,5962.957145,0,-1.357843e+09,1
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,0,1970-01-01 00:00:01.398602716,1,1,83,8074.000000,0,-1.398603e+09,0
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,2,1970-01-01 00:00:01.338638159,0,0,6,5962.957145,0,-1.338465e+09,5


### Analysis

In [21]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  int64         
 5   last_session_creation_time  12000 non-null  datetime64[ns]
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          12000 non-null  float64       
 10  adopted_user                12000 non-null  int64         
 11  usage_length                12000 non-null  float64   

In [22]:
from sklearn.model_selection import train_test_split

#set up data by separating out the labels, then split for cross validation
# data = feature_df.drop('adopted_user', axis=1)
# labels = feature_df.adopted_user

# define features and target
X = users[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'usage_length', 'email_provider']]
y = users[['adopted_user']]
# split training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
# Feature Scaling - Normalize the features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Hyperparameter Tuning

In [24]:
import sys
sys.path.append('/Users/elinorvelasquez/anaconda3/lib/python3.8/site-packages/')
#!conda install -c conda-forge nltk

from xgboost import XGBClassifier, cv, plot_importance
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# dictionary of parameters to optimize, and the range of optimization values
parameters = {
    'max_delta_step':[0,0.05],
    'scale_pos_weight':[0.95, 0.9],
    'subsample':[1,0.8],
    'learning_rate': [0.25, 0.2, 0.1],
    'min_child_weight':[0,1,2],
    'gamma':[1, 2, 3]
    }

# instantiate classifier
xg_clf = XGBClassifier(objective = 'binary:logistic', random_state = 42, silent = 1, n_estimators=10, max_depth= 8)

# RandomSearch
grid_search = GridSearchCV(param_grid=parameters, estimator=xg_clf, cv=3, scoring='accuracy')
grid_search.fit(X_train,y_train);

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[12:55:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:20] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

...

[12:55:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:55:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1

In [25]:
# Print best parameters and results
print(grid_search.best_params_)
print(grid_search.best_score_)

{'gamma': 1, 'learning_rate': 0.25, 'max_delta_step': 0.05, 'min_child_weight': 0, 'scale_pos_weight': 0.95, 'subsample': 1}
0.8628109452736319


#### Validate best parameters 

In [26]:
from sklearn.metrics import accuracy_score, mean_squared_error

# Best parameters
optimized_parameters = {'objective':'binary:logistic',
                        'n_estimators': 10,
                        'random_state': 42,
                        'silent': 0,
                        'subsample': 1, 
                        'scale_pos_weight': 0.95, 
                        'min_child_weight': 1, 
                        'max_depth': 8, 
                        'max_delta_step': 0, 
                        'learning_rate': 0.2, 
                        'gamma': 3
                       }

# Instantiate classifier
xg_clf_opt = XGBClassifier(**optimized_parameters)
xg_clf_opt.fit(X_train, y_train)

# Predict the labels
y_opt_pred = xg_clf_opt.predict(X_train)
# Score model
score = accuracy_score(y_train, y_opt_pred)

# Print accuracy
print("The resulting accuracy of the optimized classifier is {}.".format(round(score,4)))

[12:56:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:56:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The resulting accuracy of the optimized classifier is 0.8633.


In [27]:
# Predict the labels
y_opt_test = xg_clf_opt.predict(X_test)
# Score model
test_score = accuracy_score(y_test, y_opt_test)
# Print accuracy
print("The resulting accuracy of the optimized classifier for the test set is {}.".format(round(test_score,4)))

The resulting accuracy of the optimized classifier for the test set is 0.8604.


In [28]:
import sys
sys.path.append('/Users/elinorvelasquez/anaconda3/lib/python3.8/site-packages/')

# Importing TensorFlow and the Keras libraries with packages
import tensorflow
from tensorflow.keras.models import Sequential
#from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [29]:
# Initializing the Neural Network
classifier = Sequential()

In [30]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 6))

# Adding the second hidden layer
classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [31]:
# Compiling the Neural Network
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [32]:
# Fitting the Neural Network to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
804/804 [==============================] - 1s 620us/step - loss: 0.5438 - accuracy: 0.8534
Epoch 2/10
804/804 [==============================] - 1s 666us/step - loss: 0.3918 - accuracy: 0.8638
Epoch 3/10
804/804 [==============================] - 1s 634us/step - loss: 0.3997 - accuracy: 0.8578
Epoch 4/10
804/804 [==============================] - 1s 668us/step - loss: 0.3854 - accuracy: 0.8656
Epoch 5/10
804/804 [==============================] - 0s 609us/step - loss: 0.3972 - accuracy: 0.8603
Epoch 6/10
804/804 [==============================] - 0s 609us/step - loss: 0.4048 - accuracy: 0.8560
Epoch 7/10
804/804 [==============================] - 0s 613us/step - loss: 0.3889 - accuracy: 0.8643
Epoch 8/10
804/804 [==============================] - 0s 618us/step - loss: 0.3810 - accuracy: 0.8674
Epoch 9/10
804/804 [==============================] - 0s 612us/step - loss: 0.3964 - accuracy: 0.8591
Epoch 10/10
804/804 [==============================] - 0s 619us/step - loss: 0.409

In [33]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[3407    0]
 [ 553    0]]


In [34]:
# Visualizing confusion matrix
pos, neg = cm
tp, fp = pos
fn, tn = neg
print('True Positives:', tp)
print('True Negatives:', tn)
print('False Positives:', fp)
print('False Negatives:', fn)

True Positives: 3407
True Negatives: 0
False Positives: 0
False Negatives: 553


In [35]:
# Accuracy rate of confusion matrix
print('Accuracy:', (tp+tn)/(tp+tn+fp+fn))

Accuracy: 0.8603535353535353


In [36]:
users.dtypes

object_id                              int64
creation_time                 datetime64[ns]
name                                  object
email                                 object
creation_source                        int64
last_session_creation_time    datetime64[ns]
opted_in_to_mailing_list               int64
enabled_for_marketing_drip             int64
org_id                                 int64
invited_by_user_id                   float64
adopted_user                           int64
usage_length                         float64
email_provider                         int64
dtype: object

In [42]:
users_encode = users[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id', 'usage_length', 'email_provider' ]]

In [44]:
import pandas as pd
y = users[['adopted_user']]
# y = df.active_status
adopted = pd.DataFrame(y)
adopted.columns = ['adopted']
pd.concat([users_encode, adopted],
          axis=1).corr().iloc[7].sort_values(
    ascending=False)

adopted                       1.000000
usage_length                  0.086207
org_id                        0.066995
opted_in_to_mailing_list      0.008838
enabled_for_marketing_drip    0.006578
invited_by_user_id            0.000890
creation_source              -0.003465
email_provider               -0.030662
Name: adopted, dtype: float64